In [31]:
from langchain.llms import OpenAI
from dotenv import load_dotenv

load_dotenv()

llm = OpenAI()


# Basic Output

In [32]:
llm("What is the capital of France?")

'\n\nThe capital of France is Paris.'

# Introduction to chains

In [33]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain


code_prompt = PromptTemplate(
    input_variables=["language", "task"],
    template="Write a very short {language} function that can perform {task}"
)


code_chain = LLMChain(
    llm=llm,
    prompt=code_prompt,
    output_key="code"
)


result_code = code_chain({
    "language": "Python",
    "task": "addition"
})

print(result_code['code'])



def add(x, y):
    return x + y


# Multiple chains Togeather

In [34]:
validation_prompt = PromptTemplate(
    input_variables=["language", "task", "code"],
    template="""
    create testcode for the following {language} code that performs {task}: {code}    
    """
)

validation_chain = LLMChain(
    llm=llm,
    prompt=validation_prompt,
    output_key="test"
)

result_valid = validation_chain({
    "language": "Python",
    "task": "addition",
    "code": result_code['code']
})

print(result_valid['test'])


# Test 1 - Test basic addition
result = add(2, 3)
print(result) # Expected output: 5

# Test 2 - Test with negative numbers
result = add(-5, 10)
print(result) # Expected output: 5

# Test 3 - Test with decimal numbers
result = add(2.5, 3.5)
print(result) # Expected output: 6.0

# Test 4 - Test with zero
result = add(0, 5)
print(result) # Expected output: 5

# Test 5 - Test with large numbers
result = add(1000000, 500000)
print(result) # Expected output: 1500000


# Multple Chains: SequencialChain

In [35]:
from langchain.chains import SequentialChain

multiple_chains = SequentialChain(
    chains=[code_chain, validation_chain],
    input_variables=["language", "task"],
    output_variables=["code", "test"]
)

result = multiple_chains({
    "language": "Python",
    "task": "addition"
})


print(">>>> GENERATED CODE:")
print(result['code'])

print(">>>> TEST CODE:")
print(result['test'])

>>>> GENERATED CODE:


def add(x, y):
    return x + y
>>>> TEST CODE:

# Test cases for function add(x, y):

# Case 1: Both inputs are integers
assert add(5, 10) == 15

# Case 2: Both inputs are floats
assert add(2.5, 3.5) == 6.0

# Case 3: One input is integer and one input is float
assert add(5, 2.5) == 7.5

# Case 4: Both inputs are negative numbers
assert add(-5, -10) == -15

# Case 5: One input is negative number and one input is positive number
assert add(-2.5, 3.5) == 1.0

# Case 6: One input is zero
assert add(5, 0) == 5

# Case 7: Both inputs are zero
assert add(0, 0) == 0

# Case 8: One input is large integer and one input is small integer
assert add(1000000000, 5) == 1000000005

# Case 9: One input is large float and one input is small float
assert add(1000000000.5,
